## Exploration

In this Python notebook, we look at the dataset as it is using the Pandas library. We store the `transactions.csv` data into a Pandas dataframe in order to perform data exploration.

## Table of Contents:
* [Initial Impressions](#initial-impressions)
* [Data Information](#data-information)
* [Specific Data Issues](#specific-data-issues)
* [Summary](#summary)

### Initial impressions

For this section, we first look into what we can see from just a gist of the dataset.

In [1]:
import pandas as pd

path = r'..\1_DataSources\data\transactions-1.csv'
df_transaction = pd.read_csv(path)
df_transaction.head()

,trans_id,user_id,product,amount,trans_date
0,1014,1,Samsung TV,599,2022-04-01
1,1015,2,iPad,499,2022-04-05
2,1016,3,Macbook,1299,2022-04-10
3,1017,4,iPhone,999,2022-04-15
4,1018,5,Samsung TV,599,2022-04-20


In [2]:
df_transaction.tail()

,trans_id,user_id,product,amount,trans_date
79,1093,1,Kindle,89,2024-05-02
80,1094,2,Samsung TV,599,2024-05-17
81,1095,3,iPad,499,2024-05-18
82,1096,4,Macbook,1299,2024-06-01
83,1097,5,iPhone,999,2024-06-15


Looking at the first and last 5 elements of the data set, we see that the data set is sorted by `trans_id` which has a corresponding `trans_date`. We have the initial assumption that each transaction that is made creates a new row in the csv file that stands as the latest transaction. We test this assumption by checking if it is indeed sorted by `trans_id` and `trans_date`.

In [3]:
df_transaction['trans_date_datetime'] = pd.to_datetime(df_transaction['trans_date']) # adds a new column for converting the values from trans_date to a datetime data type
is_sorted = df_transaction.sort_values(by=['trans_id', 'trans_date_datetime']).equals(df_transaction) # returns True if the sorted dataframe is equal to the original dataframe
is_sorted

True

We see that since the sorted dataframe is equal to the original dataframe, then the <strong>assumption does hold true</strong> and we proceed with analyzing the dataframe with this in mind.

We also note that each `product` has its corresponding `amount`. This makes senses as most products should cost the same, but we take into consideration price changes wherein the price of a product could vary along a certain period of time. We try to see if this applies in the dataset using the code below.

In [4]:
consistency_check = df_transaction.groupby('product')['amount'].nunique() == 1 # returns True if each product has a unique value for amount
consistency_check

product
Kindle         True
Macbook       False
Samsung TV     True
iPad          False
iPhone         True
Name: amount, dtype: bool

Though this assumption holds true for the Kindle, Samsung TV, and iPhone, we do not assume that this holds true for the entire dataset as it does not hold true for the Macbook and the iPad which may have had price changes.

### Data Information

This section looks into the general shape of the data and what it is supposed to look like, assuming there are no errors in the data.

In [5]:
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   trans_id             84 non-null     int64         
 1   user_id              84 non-null     int64         
 2   product              84 non-null     object        
 3   amount               84 non-null     object        
 4   trans_date           83 non-null     object        
 5   trans_date_datetime  83 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.1+ KB


We see that `trans_id` and `user_id` are supposed to be integers and that `product` is supposed to be a string, but for `amount` which is supposed to be only float values and `trans_date` which is supposed to be only datetime values, this leads us to think that we would need to convert these columns to their correct format and that there may be errors in the data. We also notice that there are 84 non-null values for all columns but `trans_date` only has 83, leading us to think that there is a missing entry for `trans_date`.

In [6]:
df_transaction.nunique()

trans_id               84
user_id                 9
product                 5
amount                  7
trans_date             83
trans_date_datetime    83
dtype: int64

Looking at the number of unique values per column, we notice two things. First, that there are 5 values for `products` but with 7 values for `amount`. Of course, this could mean that the pricing of the product changed over time, but this does point us towards thinking that there may also be errors in these columns. Second, that the `trans_id` columns do not match the `trans_date` columns. As previously assumed and concluded before, for every `trans_id` entry, there should also be a corresponding `trans_date` entry. This serves to confirm that there may be a missing value for `trans_date` in one of the rows.

### Specific Data Issues
This section looks into data issues for each column of the dataset. We look into each entry in detail and see where the errors in the dataset lie.

It was previously mentioned that there may be problems with the `product` and `amount` columns and we look into them by looking at the unique values for both.

In [7]:
df_transaction['product'].unique()

array(['Samsung TV', 'iPad', 'Macbook', 'iPhone', 'Kindle'], dtype=object)

We see that there is nothing of note for `product`, that each element is a string and that they are all unique without any errors that are too evident.

In [8]:
df_transaction['amount'].unique()

array(['599', '499', '1299', '999', '89', '2022-10-17', '49..9'],
      dtype=object)

We see that for `amount` has 2 incorrect entries, '2022-10-17' and '49..9' which are non-float values. This coincides with the information we obtained earlier where there were 2 extra values for `amount` when compared to `product`. We then go back to one of our initial impressions where each `product` is supposed to only have one value for the `amount` to see if these errors in value caused the inconsistency, or if the Macbook and iPad still had varying prices throughout the dataset.

In [9]:
error_amount_values = ['2022-10-17', '49..9']
filtered_df = df_transaction[~(df_transaction['amount'].isin(error_amount_values))] # drops the rows with the error values
consistency_check = filtered_df.groupby('product')['amount'].nunique() == 1 # returns True if each product value has a unique amount value
consistency_check

product
Kindle        True
Macbook       True
Samsung TV    True
iPad          True
iPhone        True
Name: amount, dtype: bool

We see that the consistency in the two columns now holds true and conclude that the incorrect entries are the cause for this. Fixing this error in data should be doable, considering that we can obtain the value for `amount` given a `product`. Next, we move on to looking into the missing value for `trans_date` by looking into its non-datetime values.

In [10]:
df_transaction['trans_date_datetime'] = pd.to_datetime(df_transaction['trans_date'], errors='coerce') # converts trans_date to datetime format
non_datetime_values = df_transaction[df_transaction['trans_date_datetime'].isna()]['trans_date'].unique() # filters to see non-datetime values
non_datetime_values

array([nan], dtype=object)

We see that instead of having incorrect values similar to `amount`, the `trans_date` column has a missing value in the form of NaT. Unfortunately, since the values for `trans_date` cannot be implied by other aspects of the dataset, the best course of action here would be to drop the rows with nulll `trans_date` values despite losing valuable data. This is because we want good quality data rather than having some data that we are unsure of.

### Summary
1. `transactions-1.csv` contains a list of transactions made with their own unique primary id stored as `trans_id` which needs non-null values for `user_id`, `product`, and `trans_date`.
2. For each `product` in a transaction, there is a corresponding `amount` which stays uniform throughout the dataset.
3. The ideal datatypes for each column is as follows:
    - `trans_id`: int
    - `user_id`: int
    - `product`: string
    - `amount`: float
    - `trans_date`: datetime
4. The dataset contains 84 entries total, with 1 entry having a null value for the `trans_date` and 2 entries having invalid values for `amount`.
5. For null or invalid values in `user_id`, `product`, and `trans_date`, the best course of action is to drop the entry as there is no way to infer the value for each entry.
6. For null or invalid values in `amount`, the value can be inferred from previous entries in the dataset. (`pricing.csv` might also be used due to the filename and columns listed in the specs but it is assumed in this context that only the `transactions.csv` file was provided as the dataset)
7. For null or invalid values in `trans_id`, there might be a workaround that involves obtaining a unique id of its own by incrementing the largest value `trans_id` but comprimising the nature of the original dataset of having a pre-sorted CSV file.